In [1]:
import tensorflow as tf 
import numpy as np 
from tensorflow import keras 
import os 
import math 
import random 
import pickle 
import glob
import matplotlib.pyplot as plt 
from collections import deque 
from tensorflow.keras import layers
import time 
import scipy.io as sio

from vehicle_model_variant import Environment 
from cell_model import CellModel 
from driver_MDP import Driver_MDP 

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [2]:
drving_cycle = '../../OC_SIM_DB/OC_SIM_DB_Cycles/Highway/01_FTP72_fuds.mat'
battery_path = "../../OC_SIM_DB/OC_SIM_DB_Bat/OC_SIM_DB_Bat_nimh_6_240_panasonic_MY01_Prius.mat"
motor_path = "../../OC_SIM_DB/OC_SIM_DB_Mot/OC_SIM_DB_Mot_pm_95_145_X2.mat"
cell_model = CellModel()
# env = Environment(cell_model, drving_cycle, battery_path, motor_path, 10)
driver = Driver_MDP(0.02)

num_states = 4

In [3]:
class OUActionNoise: 
    def __init__(self, mean, std_deviation, theta=0.15, dt=1e-2, x_initial=None): 
        self.theta = theta 
        self.mean = mean 
        self.std_dev = std_deviation 
        self.dt = dt 
        self.x_initial = x_initial 
        self.reset() 
        
    def reset(self): 
        if self.x_initial is not None: 
            self.x_prev = self.x_initial 
        else: 
            self.x_prev = 0 
            
    def __call__(self): 
        x = (
             self.x_prev + self.theta * (self.mean - self.x_prev) * self.dt 
            + self.std_dev * np.sqrt(self.dt) * np.random.normal() 
        )
        self.x_prev = x 
        return x 

In [4]:
class Buffer: 
    def __init__(self, buffer_capacity=100000, batch_size=64):      
        self.buffer_capacity = buffer_capacity 
        self.batch_size = batch_size 
        self.buffer_counter = 0 
        
        self.state_buffer = np.zeros((self.buffer_capacity, num_states))
        self.action_buffer = np.zeros((self.buffer_capacity, 1))
        self.reward_buffer = np.zeros((self.buffer_capacity, 1))
        self.next_state_buffer = np.zeros((self.buffer_capacity, num_states))
        
    def record(self, obs_tuple):
        index = self.buffer_counter % self.buffer_capacity 

        self.state_buffer[index] = obs_tuple[0]
        self.action_buffer[index] = obs_tuple[1]
        self.reward_buffer[index] = obs_tuple[2]
        self.next_state_buffer[index] = obs_tuple[3]
        
        self.buffer_counter += 1 
        
    def learn(self): 
        record_range = min(self.buffer_counter, self.buffer_capacity)
        batch_indices = np.random.choice(record_range, self.batch_size)

        state_batch = tf.convert_to_tensor(self.state_buffer[batch_indices])
        action_batch = tf.convert_to_tensor(self.action_buffer[batch_indices])
        reward_batch = tf.convert_to_tensor(self.reward_buffer[batch_indices])
        reward_batch = tf.cast(reward_batch, dtype=tf.float32)
        next_state_batch = tf.convert_to_tensor(self.next_state_buffer[batch_indices])
        
        with tf.GradientTape() as tape: 
            target_actions = target_actor(next_state_batch)
            y = reward_batch + gamma * target_critic([next_state_batch, target_actions])
            critic_value = critic_model([state_batch, action_batch])
            critic_loss = tf.math.reduce_mean(tf.square(y - critic_value)) 
        critic_grad = tape.gradient(critic_loss, critic_model.trainable_variables) 
        critic_optimizer.apply_gradients(
            zip(critic_grad, critic_model.trainable_variables)
        )
        
        with tf.GradientTape() as tape: 
            actions = actor_model(state_batch)
            critic_value = critic_model([state_batch, actions])
            actor_loss = - tf.math.reduce_mean(critic_value)
        actor_grad = tape.gradient(actor_loss, actor_model.trainable_variables) 
        actor_optimizer.apply_gradients(
            zip(actor_grad, actor_model.trainable_variables)
        )
        

In [5]:
def update_target(tau): 
    new_weights = [] 
    target_variables = target_critic.weights
    for i, variable in enumerate(critic_model.weights): 
        new_weights.append(target_variables[i] * (1 - tau) + tau * variable)
    target_critic.set_weights(new_weights)
    
    new_weights = [] 
    target_variables = target_actor.weights
    for i, variable in enumerate(actor_model.weights): 
        new_weights.append(target_variables[i] * (1 - tau) + tau * variable)
    target_actor.set_weights(new_weights)
    

In [6]:
def get_actor(): 
    last_init = tf.random_uniform_initializer(minval=-0.003, maxval=0.003)
    
    inputs = layers.Input(shape=(num_states))
    inputs_batchnorm = layers.BatchNormalization()(inputs)
    
    out = layers.Dense(512, activation="relu")(inputs_batchnorm)
#     out = layers.BatchNormalization()(out)
    out = layers.Dense(512, activation="relu")(out)
#     out = layers.BatchNormalization()(out)
    outputs = layers.Dense(1, activation="sigmoid", 
                          kernel_initializer=last_init)(out)
    model = tf.keras.Model(inputs, outputs)
    return model

In [7]:
def get_critic(): 
    state_input = layers.Input(shape=(num_states))
    state_input_batchnorm = layers.BatchNormalization()(state_input)
    
    state_out = layers.Dense(16, activation="relu")(state_input_batchnorm)
#     state_out = layers.BatchNormalization()(state_out)
    state_out = layers.Dense(32, activation="relu")(state_out)
#     state_out = layers.BatchNormalization()(state_out)
    
    action_input = layers.Input(shape=(1))
    action_out = layers.Dense(32, activation="relu")(action_input)
#     action_out = layers.BatchNormalization()(action_out)
    
    concat = layers.Concatenate()([state_out, action_out]) 
    
    out = layers.Dense(512, activation="relu")(concat)
#     out = layers.BatchNormalization()(out)
    out = layers.Dense(512, activation="relu")(out)
#     out = layers.BatchNormalization()(out)
    outputs = layers.Dense(1)(out)
    
    model = tf.keras.Model([state_input, action_input], outputs)
    return model 
    

In [8]:
def policy(state, noise_object): 
    j_min = state[0][2].numpy()
    j_max = state[0][3].numpy()
    sampled_action = tf.squeeze(actor_model(state)) 
    noise = noise_object()
    sampled_action = sampled_action.numpy() + noise 
    legal_action = sampled_action * j_max 
    legal_action = np.clip(legal_action, j_min, j_max)
#     print(j_min, j_max, legal_action, noise)
    return legal_action 
    

In [9]:
def policy_epsilon_greedy(state, eps): 
    j_min = state[0][-2].numpy()
    j_max = state[0][-1].numpy()

    if random.random() < eps: 
        a = random.randint(0, 9)
        return np.linspace(j_min, j_max, 10)[a]
    else: 
        sampled_action = tf.squeeze(actor_model(state)).numpy()  
        legal_action = sampled_action * j_max 
        legal_action = np.clip(legal_action, j_min, j_max)
        return legal_action

In [10]:
std_dev = 0.2 
ou_noise = OUActionNoise(mean=0, std_deviation=0.2)

critic_lr = 0.0005 
actor_lr = 0.00025 
critic_optimizer = tf.keras.optimizers.Adam(critic_lr)
actor_optimizer = tf.keras.optimizers.Adam(actor_lr)

total_episodes = 200
gamma = 0.95 
tau = 0.001 

MAX_EPSILON = 1.0 
MIN_EPSILON = 0.01 
DECAY_RATE = 0.00002
BATCH_SIZE = 32 
DELAY_TRAINING = 5000 

In [11]:
def initialization(): 
    actor_model = get_actor() 
    critic_model = get_critic() 
    target_actor = get_actor() 
    target_critic = get_critic() 
    target_actor.set_weights(actor_model.get_weights())
    target_critic.set_weights(critic_model.get_weights())
#     actor_model.load_weights("./DDPG1_trial1/actor_model_checkpoint")
#     critic_model.load_weights("./DDPG1_trial1/critic_model_checkpoint")
#     target_actor.load_weights("./DDPG1_trial1/target_actor_checkpoint")
#     target_critic.load_weights("./DDPG1_trial1/target_critic_checkpoint")
    
    buffer = Buffer(500000, BATCH_SIZE)
    return actor_model, critic_model, target_actor, target_critic, buffer

In [12]:
def save_weights(actor_model, critic_model, target_actor, target_critic, root): 
    if not os.path.exists(root): 
        os.makedirs(root)
        
    actor_model.save_weights("./{}/actor_model.h5".format(root))
    critic_model.save_weights("./{}/critic_model.h5".format(root))
    target_actor.save_weights("./{}/target_actor.h5".format(root))
    target_critic.save_weights("./{}/target_critic.h5".format(root))
    print("model is saved..")

In [13]:
def initialization_env(driving_path, reward_factor):
    env = Environment(cell_model, driving_path, battery_path, motor_path, reward_factor)
    return env 

In [14]:
def test_agent(actor_model, reward_factor):
    test_cycle = driver.get_cycle() 
    env = initialization_env(test_cycle, reward_factor)
    
    total_reward = 0
    state = env.reset() 
    while True: 
        tf_state = tf.expand_dims(tf.convert_to_tensor(state), 0)
        action = policy_epsilon_greedy(tf_state, -1)
        next_state, reward, done = env.step(action)
        
        state = next_state 
        total_reward += reward 
        
        if done: 
            break 
        
    SOC_deviation_history = np.sum(np.abs(np.array(env.history["SOC"]) - 0.6)) 
    
    print("******************* Test is start *****************")
#     print(test_cycle)
    print('Total reward: {}'.format(total_reward), 
          "SOC: {:.4f}".format(env.SOC), 
          "Cumulative_SOC_deviation: {:.4f}".format(SOC_deviation_history), 
          "Fuel Consumption: {:.4f}".format(env.fuel_consumption))
    print("******************* Test is done *****************")
    print("")
    plt.subplot(1, 2, 1)
    plt.plot(test_cycle)
    plt.subplot(1, 2, 2)
    plt.plot(env.history["Action"])
    plt.show() 
    return env.history  
    

In [15]:
def update_reward_factor(reward_factor_temp, result_dict, thresh): 
    SOCs = result_dict["SOCs"]
    SOC_at_equilibrium = np.mean(SOCs[-10:])
    if abs(SOC_at_equilibrium - 0.6) < thresh: 
        terminal = True 
        reward_factor = reward_factor_temp 
    else: 
        terminal = False 
        reward_factor = reward_factor_temp + 5 * (0.6 - SOC_at_equilibrium)
    return reward_factor, terminal 

In [16]:
results_dict = {} 
reward_factor_temp = 1.5

driving_cycle_path = '../../OC_SIM_DB/OC_SIM_DB_Cycles/Highway/01_FTP72_fuds.mat'
driving_cycle = sio.loadmat(driving_cycle_path)
driving_cycle = driving_cycle["sch_cycle"][:, 1]
while True: 
    print("")
    print("reward factor = {}".format(reward_factor_temp))
    print("")
    
    actor_model, critic_model, target_actor, target_critic, buffer = initialization()
    
    eps = MAX_EPSILON 
    steps = 0
    
    episode_rewards = [] 
    episode_SOCs = [] 
    episode_FCs = [] 
    episode_test_history = [] 
    episode_num_test = [] 
    for ep in range(total_episodes): 
        env = initialization_env(driving_cycle, reward_factor_temp)
        
        start = time.time() 
        state = env.reset() 
        episodic_reward = 0 
        while True: 
            tf_state = tf.expand_dims(tf.convert_to_tensor(state), 0)
            action = policy_epsilon_greedy(tf_state, eps)
    #         print(action)
            next_state, reward, done = env.step(action)
            if done: 
                next_state = [0] * num_states 

            buffer.record((state, action, reward, next_state))
            episodic_reward += reward 

            if steps > DELAY_TRAINING: 
                buffer.learn() 
                update_target(tau)
                eps = MIN_EPSILON + (MAX_EPSILON - MIN_EPSILON) * np.exp(-DECAY_RATE * (steps
                                                                        -DELAY_TRAINING))

            steps += 1

            if done: 
                break 

            state = next_state 

        elapsed_time = time.time() - start 
        print("elapsed_time: {:.3f}".format(elapsed_time))
        episode_rewards.append(episodic_reward) 
        episode_SOCs.append(env.SOC)
        episode_FCs.append(env.fuel_consumption)
        
        SOC_deviation_history = np.sum(np.abs(np.array(env.history["SOC"]) - 0.6)) 
        print(
              'Episode: {}'.format(ep + 1),
              "Exploration P: {:.4f}".format(eps),
              'Total reward: {}'.format(episodic_reward), 
              "SOC: {:.4f}".format(env.SOC), 
              "Cumulative_SOC_deviation: {:.4f}".format(SOC_deviation_history), 
              "Fuel Consumption: {:.4f}".format(env.fuel_consumption), 
        )
        print("")
        
        if (ep + 1) % 10 == 0: 
#             history = test_agent(actor_model, reward_factor_temp)
            history = env.history 
            episode_test_history.append(history) 
            episode_num_test.append(ep + 1)
            
    root = "cycleOne_reward_factor{}".format(reward_factor_temp)
    save_weights(actor_model, critic_model, target_actor, target_critic, root)
    
    results_dict[reward_factor_temp] = {
        "rewards": episode_rewards, 
        "SOCs": episode_SOCs, 
        "FCs": episode_FCs, 
        "test_history": episode_test_history, 
        "test_episode_num": episode_num_test,
    }
    
    reward_factor_temp, terminal = update_reward_factor(reward_factor_temp, 
                                                        results_dict[reward_factor_temp], 
                                                        0.015) 
    if terminal: 
        break 
    
    


reward factor = 1.5

maximum steps, simulation is done ... 
elapsed_time: 16.408
Episode: 1 Exploration P: 1.0000 Total reward: -925.5790850376394 SOC: 0.9994 Cumulative_SOC_deviation: 488.2358 Fuel Consumption: 193.2254

maximum steps, simulation is done ... 
elapsed_time: 16.431
Episode: 2 Exploration P: 1.0000 Total reward: -921.1534891301686 SOC: 1.0000 Cumulative_SOC_deviation: 487.0446 Fuel Consumption: 190.5865

maximum steps, simulation is done ... 
elapsed_time: 16.776
Episode: 3 Exploration P: 1.0000 Total reward: -926.2353738718288 SOC: 1.0000 Cumulative_SOC_deviation: 490.2058 Fuel Consumption: 190.9267


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you

maximum steps, simulation is done ... 
elapsed_time: 88.253
Episode: 28 Exploration P: 0.5172 Total reward: -326.855561305813 SOC: 0.8473 Cumulative_SOC_deviation: 121.1925 Fuel Consumption: 145.0668

maximum steps, simulation is done ... 
elapsed_time: 86.974
Episode: 29 Exploration P: 0.5034 Total reward: -237.15087385459438 SOC: 0.7472 Cumulative_SOC_deviation: 67.1550 Fuel Consumption: 136.4184

maximum steps, simulation is done ... 
elapsed_time: 87.529
Episode: 30 Exploration P: 0.4901 Total reward: -249.91533238805408 SOC: 0.6740 Cumulative_SOC_deviation: 78.7089 Fuel Consumption: 131.8520

maximum steps, simulation is done ... 
elapsed_time: 86.114
Episode: 31 Exploration P: 0.4771 Total reward: -311.80549228784014 SOC: 0.6211 Cumulative_SOC_deviation: 122.0856 Fuel Consumption: 128.6772

maximum steps, simulation is done ... 
elapsed_time: 83.483
Episode: 32 Exploration P: 0.4644 Total reward: -241.0238952415473 SOC: 0.7261 Cumulative_SOC_deviation: 70.0875 Fuel Consumption: 1

C:\Users\ValueCreatorSong\Desktop\Academic\graduate_paper\degradation_model\experiment_essential_modelChange\DDPG_adaptive_rewardfactor_final\vehicle_model_variant.py:251: RuntimeWarning: invalid value encountered in double_scalars
  del_i = (1 / (2 * r_cha)) * (v_cha - (v_cha ** 2 - 4 * r_cha * p_bat) ** (0.5)) * (p_bat < 0) + (1 / (


maximum steps, simulation is done ... 
elapsed_time: 79.770
Episode: 47 Exploration P: 0.3110 Total reward: -832.6119354231329 SOC: 0.0492 Cumulative_SOC_deviation: 495.7743 Fuel Consumption: 88.9504

battery power is 4556.04963138524(+) but condition is not avail
elapsed_time: 72.247
Episode: 48 Exploration P: 0.3036 Total reward: -1742.3436841159255 SOC: -0.0001 Cumulative_SOC_deviation: 444.8426 Fuel Consumption: 75.9808

battery power is 9851.64969299313(+) but condition is not avail
elapsed_time: 74.774
Episode: 49 Exploration P: 0.2961 Total reward: -1798.9029718501974 SOC: -0.0014 Cumulative_SOC_deviation: 480.1742 Fuel Consumption: 79.5446



C:\Users\ValueCreatorSong\Desktop\Academic\graduate_paper\degradation_model\experiment_essential_modelChange\DDPG_adaptive_rewardfactor_final\vehicle_model_variant.py:252: RuntimeWarning: invalid value encountered in double_scalars
  2 * r_dis)) * (v_dis - (v_dis ** 2 - 4 * r_dis * p_bat) ** (0.5)) * (p_bat >= 0)


...


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

maximum steps, simulation is done ... 
elapsed_time: 26.458
Episode: 4 Exploration P: 0.9903 Total reward: -2959.803591380475 SOC: 1.0000 Cumulative_SOC_deviation: 486.7523 Fuel Consumption: 193.4831

maximum steps, simulation is done ... 
elapsed_time: 59.723
Episode: 5 Exploration P: 0.9638 Total reward: -2884.050920989086 SOC: 1.0000 Cumulative_SOC_deviation: 474.3228 Fuel Consumption: 188.3703

maximum steps, simulation is done ... 
elapsed_time: 58.833
Episode: 6 Exploration P: 0.9379 Total reward:

maximum steps, simulation is done ... 
elapsed_time: 59.991
Episode: 41 Exploration P: 0.3649 Total reward: -1451.8253150913265 SOC: 0.3847 Cumulative_SOC_deviation: 236.6351 Fuel Consumption: 106.9759

maximum steps, simulation is done ... 
elapsed_time: 60.002
Episode: 42 Exploration P: 0.3553 Total reward: -1181.9034683819648 SOC: 0.4519 Cumulative_SOC_deviation: 188.3462 Fuel Consumption: 111.4905

maximum steps, simulation is done ... 
elapsed_time: 60.090
Episode: 43 Exploration P: 0.3459 Total reward: -1655.8967023765074 SOC: 0.3170 Cumulative_SOC_deviation: 273.5510 Fuel Consumption: 101.2464

maximum steps, simulation is done ... 
elapsed_time: 60.370
Episode: 44 Exploration P: 0.3368 Total reward: -176.55136820470642 SOC: 0.6188 Cumulative_SOC_deviation: 9.6769 Fuel Consumption: 121.5552

maximum steps, simulation is done ... 
elapsed_time: 60.341
Episode: 45 Exploration P: 0.3280 Total reward: -186.4086522219656 SOC: 0.6059 Cumulative_SOC_deviation: 11.7903 Fuel Consumption:

maximum steps, simulation is done ... 
elapsed_time: 61.398
Episode: 82 Exploration P: 0.1251 Total reward: -325.3688733448226 SOC: 0.5688 Cumulative_SOC_deviation: 38.0677 Fuel Consumption: 109.0217

maximum steps, simulation is done ... 
elapsed_time: 61.242
Episode: 83 Exploration P: 0.1220 Total reward: -299.00967399790994 SOC: 0.5796 Cumulative_SOC_deviation: 33.1776 Fuel Consumption: 110.4541

maximum steps, simulation is done ... 
elapsed_time: 60.571
Episode: 84 Exploration P: 0.1190 Total reward: -318.8127884660092 SOC: 0.5761 Cumulative_SOC_deviation: 36.8676 Fuel Consumption: 109.2862

maximum steps, simulation is done ... 
elapsed_time: 60.602
Episode: 85 Exploration P: 0.1160 Total reward: -267.26260232379377 SOC: 0.5728 Cumulative_SOC_deviation: 27.8313 Fuel Consumption: 109.0911

maximum steps, simulation is done ... 
elapsed_time: 60.527
Episode: 86 Exploration P: 0.1131 Total reward: -281.71158477151846 SOC: 0.5749 Cumulative_SOC_deviation: 30.2564 Fuel Consumption: 10

maximum steps, simulation is done ... 
elapsed_time: 61.082
Episode: 123 Exploration P: 0.0473 Total reward: -202.87243968709274 SOC: 0.5931 Cumulative_SOC_deviation: 16.4398 Fuel Consumption: 109.4416

maximum steps, simulation is done ... 
elapsed_time: 60.832
Episode: 124 Exploration P: 0.0463 Total reward: -230.19528896837485 SOC: 0.5736 Cumulative_SOC_deviation: 21.6478 Fuel Consumption: 107.1660

maximum steps, simulation is done ... 
elapsed_time: 60.724
Episode: 125 Exploration P: 0.0453 Total reward: -259.730127487612 SOC: 0.5773 Cumulative_SOC_deviation: 26.6781 Fuel Consumption: 108.1129

maximum steps, simulation is done ... 
elapsed_time: 60.955
Episode: 126 Exploration P: 0.0444 Total reward: -303.4805870253835 SOC: 0.5640 Cumulative_SOC_deviation: 34.4645 Fuel Consumption: 107.6110

maximum steps, simulation is done ... 
elapsed_time: 61.166
Episode: 127 Exploration P: 0.0435 Total reward: -215.15697617322806 SOC: 0.5979 Cumulative_SOC_deviation: 18.3924 Fuel Consumption

maximum steps, simulation is done ... 
elapsed_time: 61.120
Episode: 164 Exploration P: 0.0221 Total reward: -169.70544354585482 SOC: 0.5891 Cumulative_SOC_deviation: 11.0365 Fuel Consumption: 106.9825

maximum steps, simulation is done ... 
elapsed_time: 60.764
Episode: 165 Exploration P: 0.0218 Total reward: -185.74239171446362 SOC: 0.5878 Cumulative_SOC_deviation: 13.8931 Fuel Consumption: 106.7850

maximum steps, simulation is done ... 
elapsed_time: 60.925
Episode: 166 Exploration P: 0.0215 Total reward: -179.3101733196951 SOC: 0.5874 Cumulative_SOC_deviation: 12.7312 Fuel Consumption: 106.9560

maximum steps, simulation is done ... 
elapsed_time: 60.943
Episode: 167 Exploration P: 0.0212 Total reward: -180.71676101439573 SOC: 0.5884 Cumulative_SOC_deviation: 12.9454 Fuel Consumption: 107.1452

maximum steps, simulation is done ... 
elapsed_time: 61.010
Episode: 168 Exploration P: 0.0209 Total reward: -186.42170635985056 SOC: 0.5908 Cumulative_SOC_deviation: 13.9122 Fuel Consumpti

In [17]:
with open("DDPG_adaptive_reward_factor.pkl", "wb") as f: 
    pickle.dump(results_dict, f, pickle.HIGHEST_PROTOCOL)

In [18]:
# results_dict